In [1]:
# useing this variable for saving models and summaries
exp_name = "RCNN_cpd_64"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.CNN_lstm_model as CNN_lstm_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp7.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((1358, 1024, 77), (1358, 1024, 1))

In [7]:
test_data.shape, test_gt.shape

((233, 1024, 77), (233, 1024, 1))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 10

pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 1

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

print s[0]*s[1], (s[0]*s[1]/resolution), (s[0]*s[1]/resolution)*resolution

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


1390592 21728 1390592


In [9]:
pos_weight.shape

(1,)

In [10]:
pos_weight

array([ 2.15172601], dtype=float32)

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [13]:
model = CNN_lstm_model.Model(n_variables = n_variables,
                        n_classes = n_classes)

In [14]:
model.build_cnn_lstm_net(pooling_stride = pooling_stride,
                         weights_shape = weights_shape,
                         bias_shape = bias_shape,
                         activation = activation,
                         lstm_n_hidden = lstm_n_hidden,
                         n_classes = n_classes,
                         learning_rate = learning_rate,
                         pos_weight = pos_weight,
                         trainable_conv = False,
                         trainable_last = True)

In [15]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'change')

In [16]:
#load the model
saver = tf.train.Saver(var_list = [
    model.weights['wc0'],model.weights['wc1'],model.weights['wc2'],
    model.bias['bc0'],model.bias['bc1'],model.bias['bc2'],
    #model.weights_rnn['out'], model.bias_rnn['out'],
])
saver.restore(sess, '../model/RCNN9001_opp/RCNN9001_opp-20000')

INFO:tensorflow:Restoring parameters from ../model/RCNN9001_opp/RCNN9001_opp-20000


In [17]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
AUC = 0.181 vs. 0.314
F1 = 0.310 vs. 0.429
Precision = 0.184 vs. 0.327
Recall = 1.000 vs. 0.621
Validation
AUC = 0.167 vs. 0.192
F1 = 0.319 vs. 0.317
Precision = 0.190 vs. 0.188
Recall = 0.994 vs. 1.000
Minibatch Loss= 2.810703
Train Batch Evaluation
AUC = 0.900 vs. 0.298
F1 = 0.879 vs. 0.343
Precision = 0.853 vs. 0.222
Recall = 0.906 vs. 0.750
Validation
AUC = 0.730 vs. 0.193
F1 = 0.708 vs. 0.317
Precision = 0.670 vs. 0.188
Recall = 0.751 vs. 1.000
Minibatch Loss= 0.232003
Train Batch Evaluation
AUC = 0.940 vs. 0.452
F1 = 0.861 vs. 0.608
Precision = 0.843 vs. 0.440
Recall = 0.881 vs. 0.985
Validation
AUC = 0.762 vs. 0.192
F1 = 0.726 vs. 0.317
Precision = 0.648 vs. 0.188
Recall = 0.825 vs. 0.999
Minibatch Loss= 0.381427
Train Batch Evaluation
AUC = 0.928 vs. 0.387
F1 = 0.892 vs. 0.602
Precision = 0.825 vs. 0.430
Recall = 0.971 vs. 1.000
Validation
AUC = 0.769 vs. 0.199
F1 = 0.722 vs. 0.317
Precision = 0.667 vs. 0.188
Recall = 0.786 vs. 1.000
Minibatch Loss= 0.379

In [18]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/RCNN_cpd_64/RCNN_cpd_64-20000'

In [ ]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [19]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.781 vs. 0.199
F1 = 0.738 vs. 0.317
Precision = 0.739 vs. 0.188
Recall = 0.736 vs. 0.997


In [20]:
probability_of_pos = nn_eval.deploy(model, sess, test_data, test_gt)

In [21]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [22]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [ ]:
1